In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from pylab import rcParams
rcParams['figure.figsize'] = 10, 8
rcParams['figure.dpi'] = 300

import torch
from torch import nn
from torch import Tensor
from torch import distributions
from torch.nn.parameter import Parameter

from sklearn import cluster, datasets, mixture
from sklearn.preprocessing import StandardScaler

In [2]:
def prox(x): #proximal mapping is the soft thresholding for l1 norm
    lam = 1
    # soft thresholding
    sub = x - lam
    zeros = sub - sub
    n_sub = torch.max(sub, zeros)
    x = torch.mul(torch.sign(x), n_sub)
    return x

def m(z, beta):
    lam = 1.0
    g = torch.norm   #g is l1 norm here
    out = (lam * g(z, p = 1, dim = 1)) + (torch.norm(z-beta, p = 2, dim = 1) ** 2)/ 2
    return out


class Mfunction_approach(torch.autograd.Function):  
                                                        
    @staticmethod                                  
    def forward(ctx, theta: Tensor, beta: Tensor):
        z = prox(beta)  
        ctx.save_for_backward(theta, beta, z) 
        out = m(theta, beta) - m(z, beta)
        return out                    
         
    @staticmethod                                 
    def backward(ctx, grad_output):
        lam = 1
        theta, beta, z= ctx.saved_tensors        
        d = grad_output.shape
        grad = grad_output.reshape([d[0],1])  
        out_1 = (lam * torch.sign(theta) + theta- beta) * grad
        out_2 = -(theta - z) * grad
        return out_1, out_2
    
    
n = 5
Kc = 100 
beta = torch.rand([n,Kc])
theta = torch.rand([n,Kc])
beta = beta.requires_grad_()
theta = theta.requires_grad_()
loss = Mfunction_approach.apply(theta, beta).sum()  
loss.backward()
#print(theta.grad, beta.grad)

y_theta = theta.clone().detach()
y_beta = beta.clone().detach()
y_theta= y_theta.requires_grad_()
y_beta = y_beta.requires_grad_()
def m_fun(theta, beta):
    return m(theta, beta) - m(prox(beta), beta)
loss_y = m_fun(y_theta, y_beta).sum()
loss_y.backward()
#print(y_theta.grad, y_beta.grad)
torch.set_printoptions(precision = 20)
#torch.isclose(beta.grad, y_beta.grad).all()
(beta.grad - y_beta.grad).sum()

tensor(0.)